##Тестовое Самокат

###Загрузка данных

In [1]:
# Чтобы использовать SQL внутри Python установим pandasql
!pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=b20ebccfe09c64cf3b067ac21e25b18ecf64bd2cde1e1f763502db08627c265f
  Stored in directory: /root/.cache/pip/wheels/68/5d/a5/edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql


In [2]:
import pandasql
import pandas as pd

In [3]:
products = pd.read_csv('/content/products.csv', encoding='MacCyrillic')
products.head()

,product_id,product,category
0,8,"Мороженое Коровка из Кореновки пломбир, ваниль...",Замороженная продукция
1,9,"Мороженое Коровка из Кореновки пломбир, шокола...",Замороженная продукция
2,12,"Мороженое Коровка из Кореновки пломбир, крем-б...",Замороженная продукция
3,16,"Мороженое Коровка из Кореновки пломбир, шокола...",Замороженная продукция
4,22,"Мороженое Коровка из Кореновки пломбир, двухсл...",Замороженная продукция


In [4]:
orders = pd.read_csv('/content/orders.csv')
orders.head(5)

,order_date,order_id,warehouse_id,user_id
0,2017-08-01,910381.0,38.0,11473.0
1,2017-08-01,918510.0,276.0,52410.0
2,2017-08-01,909015.0,208.0,10155.0
3,2017-08-01,915064.0,2.0,60635.0
4,2017-08-01,914110.0,38.0,17185.0


In [5]:
order_lines = pd.read_excel('/content/order_lines.xlsx')
order_lines.head()

,order_id,product_id,price,quantity
0,47772,1929,110.25,1
1,47772,3785,16.50,1
2,47772,3967,39.00,1
3,47772,5837,36.75,1
4,56496,2591,171.75,1


In [6]:
warehouses = pd.read_csv('/content/warehouses.csv')
warehouses.head()

,city,warehouse_id,address
0,Москва,1,"Центральная, ул. 28"
1,Санкт-Петербург,2,"Молодежная, ул. 14"
2,Москва,8,"Школьная, ул. 21"
3,Москва,10,"Лесная, ул. 27"
4,Санкт-Петербург,12,"Советская, ул. 20"


###Задание 1

Нужно на основе приложенных выгрузок сделать отчет в Power BI и отправить файл .pbix или ссылку на него.

Требования к отчету - представь, что это все данные, которые вообще есть и ты делаешь первый в компании отчет.
Твоя задача выжать из этих выгрузок максимум пользы.
Какие показатели посчитать, какие визуализации выбрать и как организовать отчет - твой выбор.

Выгрузки:
- warehouses
- products
- orders
- order_lines

В выгрузках рандомные данные, но стоит воспринимать их как фактические.
Всю обработку нужно делать внутри Power BI - исходные файлы оставь как есть.

[Ссылка на отчет](https://github.com/KirGo-91/test_tasks/blob/main/samokat/report_samokat.pbix)

###Задание 2  

Выгрузить список user_id тех пользователей,
кто купил за период 1-15 августа 2 любых корма для животных, кроме "Корм Kitekat для кошек, с кроликом в соусе, 85 г".
Отправить файл .xls

Добавляем `sqldf` для того чтобы писать SQL-запросы.

In [7]:
sqldf = lambda q: pandasql.sqldf(q, globals())

In [8]:
answer = sqldf('''
select o.user_id
from orders o
join order_lines ol
on o.order_id=ol.order_id
join products p
on ol.product_id=p.product_id
where True
and o.order_date between '2017-08-01' and '2017-08-15'
and p.product like 'Корм%'
and p.product != 'Корм Kitekat для кошек, с кроликом в соусе, 85 г'
group by o.user_id
having count(o.order_id) >=2
''')

answer

,user_id
0,71262.0
1,177315.0
2,209194.0
3,226537.0
4,236913.0


In [9]:
# сохраняем ответ в excel
answer.to_excel('users_who_bought_2_feeds.xlsx', index=False)

###Задание 3  

Напишите SQL-запрос к воображаемой БД, которая состоит из тех же 4 таблиц, имеет те же ключи и те же поля.
Запрос должен вернуть топ-5 самых часто встречающихся товаров в первых заказах пользователей в СПб за период 15-30 августа.

In [10]:
# вариант с использованием оконной функции row_number()
sqldf('''
with o as (
  select *,
    row_number() over(partition by user_id order by order_date) rn
  from orders
),
t1 as (
  select p.product
  from o
  join order_lines ol
  on o.order_id=ol.order_id
  left join products p
  on ol.product_id=p.product_id
  left join warehouses w
  on o.warehouse_id=w.warehouse_id
  where True
  and o.order_date between '2017-08-15' and '2017-08-30'
  and w.city = 'Санкт-Петербург'
  and o.rn = 1
)
select product, count(product) cnt
from t1
group by product
order by cnt desc
limit 5
''')

,product,cnt
0,"Икра трески Санта Бремор Pate, деликатесная, 90 г",8
1,"Напиток Fanta (США) Apple, газированный, 355 мл",6
2,"Йогурт Чудо питьевой, с вишней и черешней, 2,4...",5
3,"Вода Нарзан минеральная, газированная, 1,8 л",5
4,"Молоко Пискарёвское пастеризованное, 2,5%, 900...",4


In [11]:
# вариант с использованием оконной функции first_value()
answer2 = sqldf('''
select product, count(product) cnt
from (select
  o.user_id,
  o.order_id,
  first_value(o.order_id) over(partition by o.user_id order by o.order_date) first_order,
  p.product
  from orders o
  join order_lines ol
  on o.order_id=ol.order_id
  left join products p
  on ol.product_id=p.product_id
  left join warehouses w
  on o.warehouse_id=w.warehouse_id
  where True
  and o.order_date between '2017-08-15' and '2017-08-30'
  and w.city = 'Санкт-Петербург'
) t1
where order_id=first_order
group by product
order by cnt desc
limit 5
''')

answer2

,product,cnt
0,"Икра трески Санта Бремор Pate, деликатесная, 90 г",8
1,"Напиток Fanta (США) Apple, газированный, 355 мл",6
2,"Йогурт Чудо питьевой, с вишней и черешней, 2,4...",5
3,"Вода Нарзан минеральная, газированная, 1,8 л",5
4,"Молоко Пискарёвское пастеризованное, 2,5%, 900...",4


In [12]:
answer2.to_excel('top_5_most_frequently_ordered_items_in_the_first_orders.xlsx', index=False)